In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

In [3]:
spark = SparkSession.builder.appName("hw_basics").getOrCreate()

25/01/29 13:42:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
#Disabled automatic broadcast join with spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [5]:
spark

# Data Dictionary


* match_details
    * a row for every players performance in a match
* matches
    * a row for every match
* medals_matches_players
    * a row for every medal type a player gets in a match
* medals
    * a row for every medal type

           

# Data Lodaing

In [6]:
!pwd

/home/iceberg/notebooks/notebooks/homework


In [7]:
!ls /home/iceberg/data/

devices.csv  maps.csv		matches.csv  medals_matches_players.csv
events.csv   match_details.csv	medals.csv


In [8]:
!head -n 3 /home/iceberg/data/matches.csv

match_id,mapid,is_team_game,playlist_id,game_variant_id,is_match_over,completion_date,match_duration,game_mode,map_variant_id
11de1a94-8d07-4162-9f5f-d3cc753c811c,c7edbf0f-f206-11e4-aa52-24be05e24f7e,true,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,true,2016-02-22 00:00:00.000000,,,
d3643e71-3e51-43e6-a200-f4a7f306ac12,cb914b9e-f206-11e4-b447-24be05e24f7e,false,d0766624-dbd7-4536-ba39-2d890a6143a9,257a305e-4dd3-41f1-9824-dfe7e8bd59e1,true,2016-02-14 00:00:00.000000,,,


`NOTE:` We are inside the docker ecosystem which has minio, spark-iceberg, mc inside it. 

In [9]:
matches = spark.read\
              .option("header", "true")\
              .csv("/home/iceberg/data/matches.csv")  # the path where the file is present

In [10]:
matches

DataFrame[match_id: string, mapid: string, is_team_game: string, playlist_id: string, game_variant_id: string, is_match_over: string, completion_date: string, match_duration: string, game_mode: string, map_variant_id: string]

In [11]:
matches.show(3)

+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|     completion_date|match_duration|game_mode|      map_variant_id|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+
|11de1a94-8d07-416...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|         true|2016-02-22 00:00:...|          NULL|     NULL|                NULL|
|d3643e71-3e51-43e...|cb914b9e-f206-11e...|       false|d0766624-dbd7-453...|257a305e-4dd3-41f...|         true|2016-02-14 00:00:...|          NULL|     NULL|                NULL|
|d78d2aae-36e4-48a...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|  

In [12]:
match_details = spark.read\
                      .option("header", "true")\
                      .csv("/home/iceberg/data/match_details.csv")  # the path where the file is present


In [13]:
match_details

DataFrame[match_id: string, player_gamertag: string, previous_spartan_rank: string, spartan_rank: string, previous_total_xp: string, total_xp: string, previous_csr_tier: string, previous_csr_designation: string, previous_csr: string, previous_csr_percent_to_next_tier: string, previous_csr_rank: string, current_csr_tier: string, current_csr_designation: string, current_csr: string, current_csr_percent_to_next_tier: string, current_csr_rank: string, player_rank_on_team: string, player_finished: string, player_average_life: string, player_total_kills: string, player_total_headshots: string, player_total_weapon_damage: string, player_total_shots_landed: string, player_total_melee_kills: string, player_total_melee_damage: string, player_total_assassinations: string, player_total_ground_pound_kills: string, player_total_shoulder_bash_kills: string, player_total_grenade_damage: string, player_total_power_weapon_damage: string, player_total_power_weapon_grabs: string, player_total_deaths: stri

In [14]:
match_details.show(3)

25/01/29 13:43:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+
|            match_id|player_gamertag|previous_spartan_rank|spartan_rank|previous_total_xp|total_xp|previous_csr_tier|previous_csr_designation|previous_csr|previous_csr_percent_to_next_tier|previous_csr_rank|current_

In [15]:
medals_matches_players = spark.read\
				              .option("header", "true")\
				              .csv("/home/iceberg/data/medals_matches_players.csv")  # the path where the file is present


In [16]:
medals_matches_players

DataFrame[match_id: string, player_gamertag: string, medal_id: string, count: string]

In [17]:
medals_matches_players.show(3)

+--------------------+---------------+----------+-----+
|            match_id|player_gamertag|  medal_id|count|
+--------------------+---------------+----------+-----+
|009fdac5-e15c-47c...|       EcZachly|3261908037|    7|
|009fdac5-e15c-47c...|       EcZachly| 824733727|    2|
|009fdac5-e15c-47c...|       EcZachly|2078758684|    2|
+--------------------+---------------+----------+-----+
only showing top 3 rows



In [25]:
medals = spark.read\
              .option("header", "true")\
              .csv("/home/iceberg/data/medals.csv")  # the path where the file is present


In [26]:
medals

DataFrame[medal_id: string, sprite_uri: string, sprite_left: string, sprite_top: string, sprite_sheet_width: string, sprite_sheet_height: string, sprite_width: string, sprite_height: string, classification: string, description: string, name: string, difficulty: string]

In [27]:
medals.show(3)

+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+--------------+--------------------+-------------+----------+
|  medal_id|          sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|classification|         description|         name|difficulty|
+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+--------------+--------------------+-------------+----------+
|2315448068|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|          NULL|                NULL|         NULL|      NULL|
|3565441934|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|          NULL|                NULL|         NULL|      NULL|
|4162659350|https://content.h...|        750|       750|                74|     

In [28]:
maps = spark.read\
              .option("header", "true")\
              .csv("/home/iceberg/data/maps.csv")  # the path where the file is present


In [29]:
medals

DataFrame[medal_id: string, sprite_uri: string, sprite_left: string, sprite_top: string, sprite_sheet_width: string, sprite_sheet_height: string, sprite_width: string, sprite_height: string, classification: string, description: string, name: string, difficulty: string]

In [32]:
maps.show(3)

+--------------------+-------------------+--------------------+
|               mapid|               name|         description|
+--------------------+-------------------+--------------------+
|c93d708f-f206-11e...|              Urban|Andesia was the c...|
|cb251c51-f206-11e...|     Raid on Apex 7|This unbroken rin...|
|c854e54f-f206-11e...|March on Stormbreak|                NULL|
+--------------------+-------------------+--------------------+
only showing top 3 rows



In [33]:
matches.write\
    .bucketBy(16, "match_id")\
    .sortBy("match_id")\
    .saveAsTable("bucketed_matches")

Py4JJavaError: An error occurred while calling o131.saveAsTable.
: org.apache.iceberg.exceptions.NoSuchTableException: Invalid table identifier: bucketed_matches
	at org.apache.iceberg.rest.RESTSessionCatalog.checkIdentifierIsValid(RESTSessionCatalog.java:981)
	at org.apache.iceberg.rest.RESTSessionCatalog.access$100(RESTSessionCatalog.java:111)
	at org.apache.iceberg.rest.RESTSessionCatalog$Builder.<init>(RESTSessionCatalog.java:621)
	at org.apache.iceberg.rest.RESTSessionCatalog$Builder.<init>(RESTSessionCatalog.java:611)
	at org.apache.iceberg.rest.RESTSessionCatalog.buildTable(RESTSessionCatalog.java:431)
	at org.apache.iceberg.catalog.BaseSessionCatalog$AsCatalog.buildTable(BaseSessionCatalog.java:84)
	at org.apache.iceberg.rest.RESTCatalog.buildTable(RESTCatalog.java:112)
	at org.apache.iceberg.CachingCatalog$CachingTableBuilder.<init>(CachingCatalog.java:219)
	at org.apache.iceberg.CachingCatalog$CachingTableBuilder.<init>(CachingCatalog.java:214)
	at org.apache.iceberg.CachingC

In [ ]:
match_details.write\
    .bucketBy(16, "match_id")\
    .sortBy("match_id")\
    .saveAsTable("bucketed_match_details")

In [ ]:
medals_matches_players.write\
    .bucketBy(16, "match_id")\
    .sortBy("match_id")\
    .saveAsTable("bucketed_medals_matches_players")